# Audio L-Systems

The symmetry and repetition found in L-Systems hints that they could be used to compose music.  
This has been attempted a few different ways, most of which don't sound good.  

Peter Worth and Susan Stepney published a pleasing translation in their paper, ["Growing Music: musical interpretations of L-Systems"](https://pdfs.semanticscholar.org/8c2f/caaf3153779ec3e838b416cd6e6d7feecdb9.pdf) 

<div class="alert alert-block alert-danger">
<b>Alert:</b> Getting sound apps over docker containers is possible, but not trivial. The following will therefore not work in our binder. We will use the wolfram-cloud version in class, and of-course you can always clone the repository locally and run this
</div>

## Creating Rules
- Visually
  - F: move forward
  - X: do nothing
  - 2: turn counter-clockwise by an angle $\theta$
  - 4: turn clockwise by an angle $\theta$
  - 6: save location
  - 8: return to saved location
  
- Musically
  - F: play pitch
  - X: rest
  - 2: go up one note in the given key
  - 4: go down one note in the given key
  - 6: save pitch
  - 8: return to saved pitch

## Generating Sheet Music

The music rules can be written in a format very similar to the one used for visual L-Systems. Given the following set of instructions:

In [ ]:
nest[state_, rules_, iter_]:=Nest[Flatten[# /. rules] &, state, iter]
instructions = 
 nest[{"X"}, {"X" -> {"F", 6, 4, "X", 8, 6, 2, "X", 8, "F", "X"}, 
   "F" -> {"F", "F"}}, 3]

These instructions can be visualized as before:

In [ ]:
visualizeLSystemPushPop[state_, rotAngle_,drawLetters_] :=
 Module[{currentAngle = 0, currentLocation = {0, 0}, 
   currentState = {}, savedState = {}, savedAngle = 0, 
   savedLocation = {0, 0}},
  (Switch[#,
      6, savedState = {savedAngle, savedLocation, savedState},
      8, {savedAngle, savedLocation, savedState} = savedState,
      _?(Or @@ Thread[# == drawLetters] &),
      currentState = {currentState, 
        Line@{savedLocation, 
          savedLocation += {Cos@savedAngle, Sin@savedAngle}}}];
     If[# == 2 || # == 4, savedAngle += I^# rotAngle]) & /@ state;
  Graphics[Flatten@currentState, ImageSize -> 500]
  ]
  
visualizeLSystemPushPop[instructions, Pi/4, {"F"}] 

and converted to sheet music using:

In [ ]:
rawConvertToMusic[state_, playLetters_] := 
 Module[{currentAngle = 0, currentPitch = 8, currentState = {1}, 
   savedState = {}, savedAngle = 0, savedPitch = 8},
  (Switch[#,
      6, savedState = {savedAngle, savedPitch, savedState},
      8, {savedAngle, savedPitch, savedState} = savedState,
      _?(Or @@ Thread[# == playLetters] &),
      currentState = {currentState, savedPitch + savedAngle}];
     If[# == 2 || # == 4, savedAngle += I^# ]) & /@ state;
  Clip[#, {1, 10}] & /@ (Flatten@currentState)[[2 ;;]]
  ]

In [ ]:
rawConvertToMusic[instructions, {"F"}]

In [ ]:
ListPlot[rawConvertToMusic[instructions, {"F"}], Frame -> True]

We can play this using Mathematica. e.g. using a c Major scale

In [ ]:
cMajor = {None, "C3", "D3", "E3", "F3", "G3", "A3", "B3", "C4", "D4", 
   "E4", "F4", "G4", "A4", "B4", "C5"};

In [ ]:
EmitSound[
 Sound[SoundNote[cMajor[[#]], 0.5, "Cello"]] & /@ 
  rawConvertToMusic[instructions, {"F"}]]

<div class="alert alert-block alert-warning">
<b>Note: </b>
`EmitSound` will proceed to play the entire sequence unless interrupted by another `EmitSound` command. As such, we define a simple 'get-out-of-jail' card
</div>

In [ ]:
stopSound[] := EmitSound[SoundNote[0]]

## Cleaning up the Music
Our music will sound a lot better if we can combine long notes:

In [ ]:
EmitSound[
 Sound[SoundNote[cMajor[[#]], 0.5, "Cello"]] & /@ {2, 2, 2, 2}]

In [ ]:
EmitSound[Sound[SoundNote[cMajor[[#]], 2, "Cello"]] & /@ {2}]

This function counts repeated F's as held notes:

In [ ]:
combineNotes[sequence_] := 
 Module[{currentPitch = 1, tally = 1, currentCombined = {}, 
   paddedSequence},
  paddedSequence = 
   Append[sequence, 1]; (*ensures that all notes are counted*)
  
  Switch[#,
     currentPitch, tally += 1,
     _, AppendTo[currentCombined, {currentPitch, tally}]; tally = 1; 
     currentPitch = #] & /@ paddedSequence;
  currentCombined]

The functions can be combined to make a general L-System player:

In [ ]:
convertToSound[{note_, duration_}, tempo_, instrument_ : "Cello", key_ : cMajor] :=
 SoundNote[key[[note]], duration*60/tempo, instrument]
 
 playSequence[sequence_, tempo_, instrument_ : "Cello", key_ : cMajor] := 
 Module[{rawSequence, combinedSequence},
  rawSequence = rawConvertToMusic[sequence, {"F"}];
  combinedSequence = combineNotes[rawSequence];
  EmitSound[
   convertToSound[#, tempo, instrument, key] & /@ combinedSequence]]

## A Few Tunes

### Koch Snowflake

In [ ]:
kochSnowflake = 
  nest[{"F", 2, 2, "F", 2, 2, 
    "F"}, {"F" -> {"F", 4, "F", 2, 2, "F", 4, "F"}}, 2];
    
kochSnowFlakeGraphic=
With[{str = SubstitutionSystem[{"F" -> "F+F--F+F"}, "F--F--F", 3]},
 GraphicsGrid[
  Partition[
   Graphics[
      Line@AnglePath[
        StringCases[#, {"F" -> {1, 0}, "+" -> {0, Pi/3}, 
          "-" -> {0, -Pi/3}}]], ImageSize -> 500] & /@ str, 4], 
  ImageSize -> 500]]

In [ ]:
rawConvertToMusic[kochSnowflake, {"F"}]
playSequence[kochSnowflake, 120]

### Pythagoras Tree

In [ ]:
pythagorasTree = 
  nest[{"A"}, {"B" -> {"B", "B"}, "A" -> {"B", 6, 2, "A", 8, 4, "A"}},
    6];
    
visualizeLSystemPushPop[
 nest[{"A"}, {"B" -> {"B", "B"}, "A" -> {"B", 6, 2, "A", 8, 4, "A"}}, 
  6], \[Pi]/4, {"A", "B"}]

In [ ]:
rawConvertToMusic[pythagorasTree, {"A", "B"}]
playSequence[pythagorasTree /. {"A" -> "F", "B" -> "F"}, 1000]

## L-System "symphonies"

In [ ]:
kochSnowFlakeGraphic

Instead of playing only one track, we'll add multiple tracks - each at a different recursion level!

In [ ]:
cello = nest[{"F", 2, 2, "F", 2, 2, "F"}, 
    {"F" -> {"F", 4, "F", 2, 2, "F", 4, "F"}}, 1];
viola = nest[{"F", 2, 2, "F", 2, 2, "F"},
    {"F" -> {"F", 4, "F", 2, 2, "F", 4, "F"}}, 2];
violin = nest[{"F", 2, 2, "F", 2, 2, "F"},
    {"F" -> {"F", 4, "F", 2, 2, "F", 4, "F"}}, 3];

In [ ]:
violinKey = {None, "C4", "D4", "E4", "F4", "G4", "A4", "B4", "C5", 
   "D5", "E5", "F5", "G5", "A5", "B5", "C6", "D6", "E6", "F6", "G6", 
   "A6" "B6", "C7"};

violaKey = {None, "A3", "B3", "C4", "D4", "E4", "F4", "G4", "A4", 
   "B4", "C5", "D5", "E5", "F5", "G5", "A5", "B5", "C6", "D6", "E6", 
   "F6", "G6", "A6"};

celloKey = {None, "A1", "B1", "C2", "D2", "E2", "F2", "G2", "A2", 
   "B2", "C3", "D3", "E3", "F3", "G3", "A3", "B3", "C4", "D4", "E4", 
   "F4", "G4", "A4"};

We write some functions to combine notes again for the symphony

In [ ]:
combineNotesSymphony[sequence_] := 
 Module[{currentPitch = 1, beginning = 1, end = 1, 
   currentCombined = {}, paddedSequence},
  paddedSequence = 
   Append[sequence, 1]; (*ensures that all notes are counted*)
  
  Switch[#,
     currentPitch, end += 1,
     _, AppendTo[currentCombined, {currentPitch, {beginning, end}}]; 
     beginning = end; end += 1; currentPitch = #] & /@ 
   paddedSequence;
  currentCombined]
  
  sequenceSymphony[sequence_, tempo_, instrument_ : "Cello", key_ : cMajor] := 
  Module[{rawSequence, combinedSequence},
  rawSequence = rawConvertToMusic[sequence, {"F"}];
  combinedSequence = combineNotesSymphony[rawSequence];
  (convertToSound[#, tempo, instrument, key] & /@ combinedSequence)]

In [ ]:
EmitSound@Join[
  (sequenceSymphony[violin, 400, "Violin", 
    violinKey]), (sequenceSymphony[viola, 100, "Viola", 
    violaKey]), (sequenceSymphony[cello, 25, "Cello", celloKey])]